<a href="https://colab.research.google.com/github/AndrewKorostin/bot_audio_to_text/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pyTelegramBotAPI
# !pip install git+https://github.com/openai/whisper.git
# !pip install pydub
# !apt-get install -y ffmpeg
# !pip install numpy scipy
# !pip install noisereduce
# !python -m spacy download en_core_web_sm
# !python -m spacy download ru_core_news_md
# !python -m spacy download ru_core_news_sm
# !pip install transformers torch
# !pip install openai

import os
import telebot
import torch
from pydub import AudioSegment
import whisper
import noisereduce as nr
import soundfile as sf
import spacy
import re

# Подключаем Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Указываем путь к папке на Google Drive
DOWNLOAD_FOLDER = '/content/drive/My Drive/downloadsoundfiles'

API_TOKEN = '7654467470:AAG8Zqs0HpG5eAy4Id83-_Z3n3CduhkiPK0'
bot = telebot.TeleBot(API_TOKEN)

# Загружаем модель Whisper
model = whisper.load_model("medium")  # tiny, base, small, medium, large

# Загружаем модель spaCy для извлечения именованных сущностей
nlp = spacy.load("ru_core_news_sm")

# Переменная для хранения пользовательского отчёта
user_reports = {}

# Функция для классификации стиля на основе ключевых слов
def classify_style_keywords(text):
    informal_keywords = ['привет', 'как дела', 'окей', 'чувак', 'пацан', 'супер', 'эмодзи', 'плиз', 'лул']
    formal_keywords = ['уважаемый', 'с уважением', 'официально', 'пожалуйста', 'серьезно', 'уверены', 'необходимо', 'собрание']

    informal_count = sum(keyword in text.lower() for keyword in informal_keywords)
    formal_count = sum(keyword in text.lower() for keyword in formal_keywords)

    if informal_count > formal_count:
        return "Неформальный"
    elif formal_count > informal_count:
        return "Официальный"
    else:
        return "Смешанный"

# Функция для классификации приоритета на основе ключевых фраз
def classify_priority_by_keywords(text):
    high_priority_keywords = ['срочно', 'неотложно', 'важно', 'экстренно', 'сейчас', 'сегодня', 'щас', 'вечером', 'обязательно', 'завтра', 'срочный']
    medium_priority_keywords = ['нужно', 'следует', 'желательно', 'неделю', 'неделя']
    low_priority_keywords = ['по возможности', 'если есть время', 'потом', 'когда удобно', 'бы', 'не плохо']

    high_count = sum(keyword in text.lower() for keyword in high_priority_keywords)
    medium_count = sum(keyword in text.lower() for keyword in medium_priority_keywords)
    low_count = sum(keyword in text.lower() for keyword in low_priority_keywords)

    if high_count > medium_count and high_count > low_count:
        return "Высокий"
    elif medium_count > high_count and medium_count > low_count:
        return "Средний"
    else:
        return "Низкий"

# Функция для извлечения имен из текста с помощью spaCy
def extract_names_from_text_spacy(text):
    doc = nlp(text)
    names = [ent.text for ent in doc.ents if ent.label_ == "PER"]
    return names

# Функция для выделения задач на основе ключевых фраз
def extract_tasks(text):
    tasks = []
    task_keywords = [
        ('делает', 'отчет'),
        ('готовит', 'презентацию'),
        ('делает', 'презентацию'),
        ('анализирует', 'данные'),
        ('подготавливает', 'документ'),
        ('пишет', 'отчет'),
        ('занимается', 'проектом'),
        ('отвечает', 'на вопросы'),
        ('провести', 'проверку'),
        ('проводит', 'встречу')
    ]

    for verb, task in task_keywords:
        if verb in text and task in text:
            tasks.append(task)

    return tasks

# Функция для обработки голосовых сообщений
@bot.message_handler(content_types=['voice'])
def handle_voice_message(message):
    # Получаем файл аудиосообщения
    file_info = bot.get_file(message.voice.file_id)
    file_name = f"{message.voice.file_id}.ogg"

    # Получаем ID пользователя из сообщения
    user_id = message.from_user.id
    print(f"ID пользователя: {user_id}")

    # Скачиваем файл
    downloaded_file = bot.download_file(file_info.file_path)

    # Записываем файл в целевую папку
    file_path = os.path.join(DOWNLOAD_FOLDER, file_name)
    with open(file_path, 'wb') as new_file:
        new_file.write(downloaded_file)

    # Конвертируем ogg в wav
    audio = AudioSegment.from_file(file_path, format='ogg')

    # Сохраняем аудио в формат wav для дальнейшей обработки
    wav_file_path = file_path.replace('.ogg', '.wav')
    audio.export(wav_file_path, format='wav')

    # Применяем шумоподавление с использованием noisereduce
    data, sample_rate = sf.read(wav_file_path)  # Чтение аудио с использованием soundfile
    reduced_noise_data = nr.reduce_noise(y=data, sr=sample_rate)

    # Сохраняем обработанное аудио
    noise_reduced_file_path = wav_file_path.replace('.wav', '_nr.wav')
    sf.write(noise_reduced_file_path, reduced_noise_data, sample_rate)

    # Выполняем транскрипцию с помощью Whisper
    result = model.transcribe(noise_reduced_file_path)
    transcript_text = result["text"]

    # Извлекаем имена из транскрибированного текста
    names = extract_names_from_text_spacy(transcript_text)

    # Извлекаем задачи
    tasks = extract_tasks(transcript_text)

    # Определение стиля текста на основе ключевых слов
    text_style = classify_style_keywords(transcript_text)

    # Определение приоритета текста на основе ключевых слов
    text_priority = classify_priority_by_keywords(transcript_text)

    # Отправляем транскрибированный текст обратно пользователю
    bot.reply_to(message, f"Транскрипция аудиосообщения:\n{transcript_text}")

    # Если имена найдены, отправляем задачи
    if names and tasks:
        for i in range(min(len(names), len(tasks))):
            bot.reply_to(message, f"Исполнительное лицо: {names[i]}\nЗадача: {tasks[i]}\nСтиль сообщения: {text_style}\nПриоритет: {text_priority}")
    else:
        bot.reply_to(message, f"Не удалось выделить исполнителей или задачи.")

    # Удаляем временные файлы, чтобы очистить пространство на диске
    # os.remove(file_path)
    # os.remove(wav_file_path)
    # os.remove(noise_reduced_file_path)

# Запускаем бота
if __name__ == '__main__':
    bot.polling(none_stop=True)

Mounted at /content/drive


100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 107MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

ID пользователя: 5197179751


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!pip install pyTelegramBotAPI
!pip install git+https://github.com/openai/whisper.git
!pip install pydub
!apt-get install -y ffmpeg
!pip install numpy scipy
!pip install noisereduce
!python -m spacy download en_core_web_sm
!python -m spacy download ru_core_news_md
!python -m spacy download ru_core_news_sm
!pip install transformers torch
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 17.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-_ehhcf77
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-_ehhcf77
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803669 sha256=2092029ae10a864ef8c7df282ecc723ccc5816f54a34827e422e6b430938b2cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ce5u3z5l/wheels/8b/6c/d0/62